In [12]:
import os
from glob import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter


import pickle
import gc

from tqdm.notebook import tqdm
#tqdm.pandas()

In [13]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, make_scorer

In [14]:
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb

In [15]:
DATA = os.path.join('.', 'data')
DATA_OWN = os.path.join('.', 'data_own')
CLICKSTREAM = 'alfabattle2_abattle_clickstream'
MODELS = os.path.join('.', 'models')
UTILS = os.path.join('.', 'utils')
SUBM = os.path.join('.', 'submissions')

## load data, features

In [16]:
load_dtypes = pickle.load(open(os.path.join(UTILS, 'load_dtypes.pkl'), 'rb'))

In [17]:
using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))
len(using_features)

91

In [18]:
load_col = list(using_features)
load_col.extend(['client_pin', 'timestamp', 'multi_class_target'])

In [19]:
%%time
data_train = pd.read_csv(os.path.join(DATA_OWN, 'data_train.csv'), parse_dates=['timestamp'], usecols=load_col, dtype=load_dtypes)
#data_train = pd.read_csv(os.path.join(DATA_OWN, 'data_train.csv'), usecols=load_col, dtype=load_dtypes)
data_train.head()

Wall time: 2min 9s


,client_pin,timestamp,multi_class_target,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,...,relations_time_past_targ_main_screen,relations_time_past_targ_statement,relations_time_past_targ_credit_info,relations_time_past_targ_own_transfer,relations_time_past_targ_mobile_recharge,relations_time_past_targ_phone_money_transfer,relations_time_past_targ_card2card_transfer,relations_time_past_targ_chat,relations_time_past_targ_card_recharge,relations_time_past_targ_invest
0,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen,0.133333,0.380952,0.000000,0.000000,0.066667,0.142857,0.276190,...,1.307955,0.163198,0.000000,0.000000,1.046685,0.341562,1.533734,0.000000,0.000000,0.000000
1,5f16c0ab27a806fd08db3122921adf3a,2020-03-21 12:59:34,invest,0.389706,0.573529,0.000000,0.007353,0.000000,0.000000,0.000000,...,0.032518,2.121950,0.000000,0.706016,0.000000,0.000000,0.000000,1.837007,0.000000,1.374358
2,ec868fc2b388293cf10e18ee9518d72f,2020-01-24 18:18:55,statement,0.116279,0.465116,0.000000,0.046512,0.279070,0.069767,0.000000,...,0.280475,0.009147,0.000000,0.421385,1.205900,0.271085,0.000000,0.234015,0.000000,0.000000
3,91f55a33d7502c1a1fa5da7ff2f7b648,2020-03-15 19:50:23,main_screen,0.457831,0.449799,0.000000,0.004016,0.032129,0.008032,0.024096,...,6.314792,0.452830,0.000000,0.575543,0.363213,1.093542,0.614683,0.580554,0.530097,1.346742
4,3ef1020bda95ce7836d2680fa553ecb7,2020-02-20 07:56:58,main_screen,0.600000,0.066667,0.266667,0.000000,0.000000,0.000000,0.000000,...,0.107516,0.446540,0.446022,0.000000,0.000000,0.000000,0.000000,0.000000,0.371353,0.000000


In [20]:
(data_train.isnull().values.any())

False

In [21]:
target = data_train.multi_class_target
data_train.drop('multi_class_target', axis = 1, inplace = True)

In [22]:
%%time
X_train, X_val, y_train, y_val = train_test_split(data_train, target, test_size=0.33, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

Wall time: 9.53 s


((3393784, 93), (1671566, 93), (3393784,), (1671566,))

In [ ]:
#using_features = pickle.load(open(os.path.join(DATA_OWN, 'using_features.pkl'), 'rb'))
#len(using_features)

In [ ]:
#f1 = make_scorer(f1_score , average='macro')

## check classifiers with check time of work

In [ ]:
#clf_sgd_log   = SGDClassifier(loss = 'log', class_weight='balanced', n_jobs=-1)

#clf_knn  = KNeighborsClassifier()
#clf_svc  = SVC()
#clf_gaus = GaussianNB()

In [ ]:
%%time
#clf_sgd_hinge = SGDClassifier(loss = 'hinge', class_weight='balanced', n_jobs=-1)
clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(X_train[using_features], y_train)
pred_sgd_hinge = clf_sgd_hinge.predict(X_val[using_features])
print(len(set(pred_sgd_hinge)), set(pred_sgd_hinge))
print(f1_score(y_val, pred_sgd_hinge, average  = 'micro'))

In [ ]:
#%%time
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', class_weight='balanced', n_jobs=-1)
#clf_sgd_perc  = SGDClassifier(loss = 'perceptron', n_jobs=-1)
#clf_sgd_perc.fit(X_train[using_features], y_train)
#pred_sgd_perc = clf_sgd_perc.predict(X_val[using_features])
#rint(len(set(pred_sgd_perc)), set(pred_sgd_perc))
#rint(f1_score(y_val, pred_sgd_perc, average  = 'micro'))

In [ ]:
%%time
# 9 min
# 91 - 1h 10min 53s

clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(X_train[using_features], y_train)
pred_mlp = clf_mlp.predict(X_val[using_features])
print(len(set(pred_mlp)), set(pred_mlp))
print(f1_score(y_val, pred_mlp, average  = 'micro'))

In [ ]:
%%time
# 81 - 8min 23s
# 91 - 8min 40s

clf_lr = LogisticRegression(n_jobs = -1, verbose = 1) #‘liblinear’,
clf_lr.fit(X_train[using_features], y_train)
pred_lr = clf_lr.predict(X_val[using_features])
print(len(set(pred_lr)), set(pred_lr))
print(f1_score(y_val, pred_lr, average  = 'micro'))

In [ ]:
%%time
# 81 - 17min 29s
# 91 - 4h 4min 35s
clf_lrsvc = LinearSVC(verbose = 1) # loss = ‘hinge’
clf_lrsvc.fit(X_train[using_features], y_train)
pred_lrsvc = clf_lrsvc.predict(X_val[using_features])
print(len(set(pred_lrsvc)), set(pred_lrsvc))
print(f1_score(y_val, pred_lrsvc, average  = 'micro'))

In [ ]:
%%time 
# 81 - 5min 38s
# 91 - 5min 21s

clf_lgbm = LGBMClassifier( silent = False) # loss = ‘hinge’
clf_lgbm.fit(X_train[using_features], y_train)
pred_lgbm = clf_lgbm.predict(X_val[using_features])
print(len(set(pred_lgbm)), set(pred_lgbm))
print(f1_score(y_val, pred_lgbm, average  = 'micro'))

## XGBoost

In [ ]:
target_encoder = OrdinalEncoder().fit(y_train.values.reshape(-1, 1))
y_train_int = target_encoder.transform(y_train.values.reshape(-1, 1))
y_val_int   = target_encoder.transform(y_val.values.reshape(-1, 1))

In [ ]:
pickle.dump(target_encoder, open(os.path.join(UTILS, 'oe_target.pkl'), 'wb'))

In [ ]:
%%time
xgb_params = {
    'tree_method':'gpu_hist',
    "objective": "multi:softmax",
    "eta": 0.3,
    "num_class": 10,
    #"max_depth": 10,
    #"nthread": 4,
    "eval_metric": "mlogloss",
    #"print.every.n": 10
    "verbosity": 2,
}

part_size = int(X_train.shape[0] / 4)
progress_result = {}
total_progress = []
Deval = xgb.DMatrix(X_val[using_features], label=y_val_int)


for part in range(4):
    #print(part*part_size, (part+1)*part_size)
    print('part ' + str(part) + '  eta: ' +  str(xgb_params['eta']))
    
    if part != 3:
        Dtrain = xgb.DMatrix(X_train[using_features][part * part_size : (part + 1) * part_size], 
                             label = y_train_int[part * part_size : (part + 1) * part_size])
    else:
        Dtrain = xgb.DMatrix(X_train[using_features][part * part_size : ], 
                             label = y_train_int[part * part_size : ])  
        
    eval_sets = [(Dtrain, 'train'), (Deval, 'eval')]
    if part == 0:
        model = xgb.train(xgb_params, Dtrain, 
                           num_boost_round = 50,
                           evals = eval_sets,        
                           #early_stopping_rounds = 10, 
                           evals_result = progress_result, 
                         )
    else:
        model = xgb.train(xgb_params, Dtrain, 
                           num_boost_round = 50,
                           xgb_model = model,
                           evals = eval_sets,        
                           #early_stopping_rounds = 10, 
                           evals_result = progress_result,
                         )
    
    total_progress.append(progress_result)
    
    ##
    ##
    ## THIS PART SHOULD BE CHECKED IN SANDBOX
    ## DOWNGREADE ETA GIVE ONLY 0.666830 ON 
    ## num_boost_round = 30 AND START ETA 0.5
    ## train-mlogloss:0.79863 eval-mlogloss:0.80901
    ## better to start with eta 0.3
    ##
    ##
    #if np.argmin(progress_result['eval']['mlogloss']) < (len(progress_result['eval']['mlogloss']) - 6 - 1):
    #    #break
    #    xgb_params.update({'eta': xgb_params['eta'] / 2}) #part
    #    print('eta downgraded to ' + str(xgb_params['eta']))
    
    # without update/refresh parameters getting 0.669192
    #xgb_params.update({#'process_type': 'update',    # 0.66732872
                       #'updater'     : 'refresh',
                       #'refresh_leaf': True
    #                  })
    
print('done!')

In [ ]:
pred_xgb = model.predict(Deval)
print(f1_score(y_val_int, pred_xgb, average  = 'micro'))

In [30]:
#%%time
clf_cb = CatBoostClassifier(task_type="GPU", #with gpu: 5 min vs 5 hour
                            devices='0'
                           )
clf_cb.fit(X_train[using_features], y_train)

Learning rate set to 0.340834
0:	learn: 1.5160972	total: 440ms	remaining: 7m 19s
1:	learn: 1.3859161	total: 803ms	remaining: 6m 40s
2:	learn: 1.2906854	total: 1.11s	remaining: 6m 8s
3:	learn: 1.2259941	total: 1.43s	remaining: 5m 56s
4:	learn: 1.1814506	total: 1.72s	remaining: 5m 43s
5:	learn: 1.1485482	total: 2.03s	remaining: 5m 36s
6:	learn: 1.1243474	total: 2.33s	remaining: 5m 31s
7:	learn: 1.0966035	total: 2.66s	remaining: 5m 29s
8:	learn: 1.0830440	total: 2.96s	remaining: 5m 25s
9:	learn: 1.0682029	total: 3.3s	remaining: 5m 26s
10:	learn: 1.0581422	total: 3.6s	remaining: 5m 23s
11:	learn: 1.0440352	total: 3.93s	remaining: 5m 23s
12:	learn: 1.0310178	total: 4.24s	remaining: 5m 22s
13:	learn: 1.0223563	total: 4.54s	remaining: 5m 19s
14:	learn: 1.0135466	total: 4.9s	remaining: 5m 21s
15:	learn: 1.0068663	total: 5.26s	remaining: 5m 23s
16:	learn: 1.0000155	total: 5.57s	remaining: 5m 22s
17:	learn: 0.9935757	total: 5.87s	remaining: 5m 20s
18:	learn: 0.9902724	total: 6.18s	remaining: 5m 

In [31]:
pred_cb = clf_cb.predict(X_val[using_features])
print(set(pred_cb.reshape(-1)))
print(f1_score(y_val, pred_cb, average  = 'micro'))

{'own_transfer', 'chat', 'invest', 'card2card_transfer', 'mobile_recharge', 'statement', 'phone_money_transfer', 'main_screen', 'card_recharge', 'credit_info'}
0.6702636928485025


### saw how many give us stack

In [ ]:
%%time
pred_stack = [''] * pred_sgd_hinge.shape[0]
collisions = 0
for idx in range(pred_sgd_hinge.shape[0]):
    cnt = Counter([
                   pred_sgd_hinge[idx], 
                   #pred_mlp[idx], 
                   pred_lr[idx], 
                   pred_lrsvc[idx]
               ]).most_common()
    pred_stack[idx] = cnt[0][0]
    
    if len(cnt) > 1 and cnt[0][1] == cnt[1][1]:
        collisions += 1
    #    print('terrible!')
    #    print(cnt[0], cnt[1])

In [ ]:
print(f1_score(y_val, pred_stack, average  = 'micro'), (collisions/pred_sgd_hinge.shape[0]))

In [ ]:
stack_df = pd.DataFrame({'SGD':pred_sgd_hinge, 'logit':pred_lr, 'linear_svc':pred_lrsvc})
cat_ftrs = ['SGD', 'logit', 'linear_svc']

In [ ]:
#cb_stack = CatBoostClassifier(loss_function='Logloss', cat_features=cat_ftrs)
cb_stack = CatBoostClassifier(cat_features=cat_ftrs)
cb_stack.fit(stack_df, y_val)
pred_stack_cb = cb_stack.predict(X_val[using_features])
print(len(set(pred_stack_cb)), set(pred_stack_cb))
print(f1_score(y_val, pred_stack_cb, average  = 'micro'))

In [ ]:
pred_stack_cb = cb_stack.predict(stack_df)
print(len(set(pred_stack_cb[:, 0])), set(pred_stack_cb[:, 0]))
print(f1_score(y_val, pred_stack_cb[:, 0], average  = 'micro'))

In [ ]:
#dataset = cb.Pool(X, y, cat_features=['d'])

## study best models on full data

In [ ]:
%%time
# 91 - 45min 1s ????

clf_sgd_hinge = SGDClassifier(loss = 'hinge', n_jobs=-1)
clf_sgd_hinge.fit(data_train[using_features], target)

In [ ]:
%%time
# 91 - 1h 28min 38s
clf_mlp  = MLPClassifier((200, 50), learning_rate = 'adaptive', activation='logistic', verbose = True)
clf_mlp.fit(data_train[using_features], target)

In [ ]:
%%time
# 81 - 13min 29s
# 91 - 1h 13min 58s
clf_lr = LogisticRegression(n_jobs = -1, verbose = 2)
clf_lr.fit(data_train[using_features], target)

In [ ]:
%%time
#81 - 27min 9s

#clf_lrsvc = LinearSVC(verbose = True) # loss = ‘hinge’
#clf_lrsvc.fit(data_train[using_features], target)

In [ ]:
clf_lgbm = LGBMClassifier( silent = False) # loss = ‘hinge’
clf_lgbm.fit(data_train[using_features], target)

In [32]:
#%%time
clf_cb = CatBoostClassifier(task_type="GPU", #with gpu: 10 min vs ?? hour
                            devices='0'
                           )
clf_cb.fit(data_train[using_features], target)

Learning rate set to 0.369849
0:	learn: 1.4870261	total: 600ms	remaining: 9m 59s
1:	learn: 1.3692881	total: 1.09s	remaining: 9m 4s
2:	learn: 1.2745629	total: 1.56s	remaining: 8m 39s
3:	learn: 1.2141961	total: 2.02s	remaining: 8m 22s
4:	learn: 1.1615249	total: 2.44s	remaining: 8m 5s
5:	learn: 1.1330374	total: 2.91s	remaining: 8m 1s
6:	learn: 1.1087398	total: 3.38s	remaining: 7m 59s
7:	learn: 1.0885528	total: 3.88s	remaining: 8m 1s
8:	learn: 1.0694524	total: 4.4s	remaining: 8m 4s
9:	learn: 1.0533579	total: 4.83s	remaining: 7m 58s
10:	learn: 1.0441554	total: 5.31s	remaining: 7m 57s
11:	learn: 1.0357919	total: 5.79s	remaining: 7m 56s
12:	learn: 1.0257256	total: 6.26s	remaining: 7m 55s
13:	learn: 1.0135420	total: 6.83s	remaining: 8m 1s
14:	learn: 1.0075711	total: 7.31s	remaining: 7m 59s
15:	learn: 1.0026405	total: 7.83s	remaining: 8m 1s
16:	learn: 0.9962476	total: 8.38s	remaining: 8m 4s
17:	learn: 0.9894886	total: 8.84s	remaining: 8m 2s
18:	learn: 0.9802161	total: 9.3s	remaining: 8m
19:	lea

In [ ]:
%%time
xgb_params = {
    'tree_method':'gpu_hist',
    "objective": "multi:softmax",
    "eta": 0.3,
    "num_class": 10,
    #"max_depth": 10,
    #"nthread": 4,
    "eval_metric": "mlogloss",
    #"print.every.n": 10
    "verbosity": 2,
}

target_int = target_encoder.transform(target.values.reshape(-1, 1))
part_size = int(data_train.shape[0] / 4)

#progress_result = {}
#total_progress = []


for part in range(4):
    #print(part*part_size, (part+1)*part_size)
    print('part ' + str(part) + '  eta: ' +  str(xgb_params['eta']))
    
    if part != 3:
        Dtrain = xgb.DMatrix(data_train[using_features][part * part_size : (part + 1) * part_size], 
                             label = target_int[part * part_size : (part + 1) * part_size])
    else:
        Dtrain = xgb.DMatrix(data_train[using_features][part * part_size : ], 
                             label = target_int[part * part_size : ])  
        
    eval_sets = [(Dtrain, 'train'), (Deval, 'eval')]
    if part == 0:
        model_xgb = xgb.train(xgb_params, Dtrain, 
                           num_boost_round = 50,
                           #early_stopping_rounds = 10, 
                         )
    else:
        model_xgb = xgb.train(xgb_params, Dtrain, 
                           num_boost_round = 50,
                           xgb_model = model_xgb,
                           #early_stopping_rounds = 10, 
                         )
    
#    total_progress.append(progress_result)
    
    ##
    ##
    ## THIS PART SHOULD BE CHECKED IN SANDBOX
    ## DOWNGREADE ETA GIVE ONLY 0.666830 ON 
    ## num_boost_round = 30 AND START ETA 0.5
    ## train-mlogloss:0.79863 eval-mlogloss:0.80901
    ## better to start with eta 0.3
    ##
    ##
    #if np.argmin(progress_result['eval']['mlogloss']) < (len(progress_result['eval']['mlogloss']) - 6 - 1):
    #    #break
    #    xgb_params.update({'eta': xgb_params['eta'] / 2}) #part
    #    print('eta downgraded to ' + str(xgb_params['eta']))
    
    # without update/refresh parameters getting 0.669192
    #xgb_params.update({#'process_type': 'update',    # 0.66732872
                       #'updater'     : 'refresh',
                       #'refresh_leaf': True
    #                  })
    
print('done!')

## save models

In [ ]:
%%time
pickle.dump(clf_sgd_hinge, open(os.path.join(MODELS, 'clf_sgd.pkl'), 'wb'))
pickle.dump(clf_mlp, open(os.path.join(MODELS, 'clf_mlp.pkl'), 'wb'))
pickle.dump(clf_lr, open(os.path.join(MODELS, 'clf_lr.pkl'), 'wb'))
pickle.dump(clf_lgbm, open(os.path.join(MODELS, 'clf_lgbm.pkl'), 'wb'))

#lgb.save(clf_lgbm, os.path.join(MODELS, 'clf_lgbm.lgb'), num_iteration = NULL)
#clf_lgbm.save_model(os.path.join(MODELS, 'clf_lgbm.txt'))

#pickle.dump(clf_lrsvc, open(os.path.join(MODELS, 'clf_lrsvc.pkl'), 'wb'))
#pickle.dump(clf_svc, open(os.path.join(MODELS, 'clf_svc.pkl'), 'wb'))
#pickle.dump(clf_rf,  open(os.path.join(MODELS, 'clf_rf.pkl'),  'wb'))


pickle.dump(clf_lgbm, open(os.path.join(MODELS, 'clf_lgbm.pkl'), 'wb'))

In [ ]:
model_xgb.save_model(os.path.join(MODELS, 'clf_xgb.json'))

In [33]:
clf_cb.save_model(os.path.join(MODELS, 'clf_cb.cbm'), format='cbm')